# National Weather Service API program

### Importing packgaes we need

In [14]:
# Pandas to manipulate data
import pandas as pd

# Numpy - primarily for vectorization
import numpy as np

# To make REST calls
import requests

# os for seeking in the file
import os

### Reading the file now.

Following intruction: <br>
"Given a list of three geographic US lat / long values in a text file, in this format (you should manually put these three lines into the text file with a text editor)" 


In [2]:
# Reading the file
latlong = pd.read_csv("LatLong.csv", header=None, names=["Latitude" , "Longitude"])

### Taking a peek to confirm read went right

In [3]:
latlong

,Latitude,Longitude
0,38.2527° N,85.7585° W
1,42.3601° N,71.0589° W
2,39.1031° N,84.5120° W


### Making the conversion from Degrees Decimal Minutes (DDM) to Decimal Degrees (DD)

Note: We can convert using LatLon package, but here we are using barebones code. (pip install LatLon)

In [4]:
# Simply keeping digit before and after the decimal, essentially stripping everything else except for float value.
latlong['Latitude_stripped'] = latlong['Latitude'].str.extract(r'(\d+.\d+)').astype('str')
latlong['Longitude_stripped'] = latlong['Longitude'].str.extract(r'(\d+.\d+)').astype('str')

# Additional given how Latitude and Longitude conversion works we need to prepend negative sign
latlong['Longitude_stripped'] = '-' + latlong['Longitude_stripped']

### Looking at the dataset again to confirm strip went right

In [5]:
latlong

,Latitude,Longitude,Latitude_stripped,Longitude_stripped
0,38.2527° N,85.7585° W,38.2527,-85.7585
1,42.3601° N,71.0589° W,42.3601,-71.0589
2,39.1031° N,84.5120° W,39.1031,-84.5120


### Setting the endpoint start string so we can change in future if needed

In [6]:
# setting our endpoint for the request which we will append with Latitude and Longitude
URL_endpoin_start_string = "https://api.weather.gov/points/"

### Writing a vectorized function to save time and keep to code succint/redable.

In [7]:
# Using numpy vectorize decorator to vectorize our custom function.
@np.vectorize
def get_location_temperature(lat, long):
    # creating a url with Latitude and Longitude to make the REST call
    URL = URL_endpoin_start_string + lat + "," + long
    
    # Making a request now
    try:
        req_first = requests.get(url = URL)
    except Exception as e:
        print("Something went wrong, refer: ", e)
    
    # Getting json for the request response we just made
    req_first_data = req_first.json()
    
    # Getting the URL which will be used to make the second call
    forecast_url = req_first_data["properties"]["forecast"]
    
    # Making the second call to get the data we need.
    try:
        req_second = requests.get(url = forecast_url)
    except Exception as e:
        print("Something went wrong, refer: ", e)
    
    # Getting json for the request response we just made
    req_second_data = req_second.json()
    
    # Getting the Wewdnesday Night temperature by paring the json.
    wednesday_temprature = req_second_data["properties"]["periods"][6]["temperature"]
    
    # Returning the row
    return wednesday_temprature

### Calling our vectorized function to get the temperature for each Latitude and Longitude
(Note: if the exposed API is busy it may not behave well, just rerun the following for anything unexpected.)

In [8]:
latlong["Wednesday_night_temp"] = get_location_temperature(latlong["Latitude_stripped"] ,latlong["Longitude_stripped"])

### Taking a look at data frame and Wednesday night temperatures

In [9]:
latlong

,Latitude,Longitude,Latitude_stripped,Longitude_stripped,Wednesday_night_temp
0,38.2527° N,85.7585° W,38.2527,-85.7585,94
1,42.3601° N,71.0589° W,42.3601,-71.0589,61
2,39.1031° N,84.5120° W,39.1031,-84.5120,94


## Writing to a local text file in the required format

In [10]:
Wednesday_night_temp_list = latlong["Wednesday_night_temp"].values.tolist()

In [17]:
with open('WednesdayNightTemperature.txt', 'w') as filehandle:
    # for looping through each value in the list
    for each in Wednesday_night_temp_list:
            filehandle.writelines("%s," % each)
    filehandle.seek(filehandle.tell() - 1, os.SEEK_SET) 
    filehandle.truncate()

### Additional content 

### Taking a quick look at the performace (seems pretty scalable for large number of rows as well)

In [13]:
%%timeit
latlong["Wednesday_night_temp"] = get_location_temperature(latlong["Latitude_stripped"] ,latlong["Longitude_stripped"])

364 ms ± 9.32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
